# Dataset Summary and Objective

## 1. Dataset Fields:
- **Name:** Patient's name (can be anonymized or excluded for modeling).
- **Age:** Patient's age.
- **Gender:** Patient's gender (male/female).
- **Blood Type:** Patient's blood type (e.g., A, B, AB, O).
- **Medical Condition:** Patient's medical issue (e.g., cancer, obesity, diabetes).
- **Date of Admission:** The date the patient was admitted to the hospital.
- **Doctor:** The attending doctor.
- **Hospital:** Hospital information.
- **Insurance Provider:** Patient's insurance company.
- **Billing Amount:** Total hospital expenses.
- **Room Number:** Hospital room number.
- **Admission Type:** Type of hospital admission (e.g., urgent, elective).
- **Discharge Date:** The date the patient was discharged.
- **Medication:** Prescribed medication.
- **Test Results:** Health test results (Normal, Inconclusive, Abnormal).

## 2. Project Objective:
- Build a classification model to predict test results (Normal, Inconclusive, Abnormal).
- Prioritize critical health cases by predicting health conditions early.
- Analyze the impact of insurance costs, medication, and admission type on test results.

## 3. Machine Learning Methods to Apply:
- **Logistic Regression:** A simple and explainable classification model.
- **Decision Trees:** To identify the most important features.
- **Support Vector Machines (SVM):** For learning complex classification boundaries.

## 4. Expected Outcomes:
- Develop a classification model to predict critical health conditions effectively.
- Improve the accuracy of test result predictions to support clinical decision-making.
- Provide insights to optimize medical costs and enhance healthcare outcomes.


# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import os
warnings.filterwarnings("ignore")


# Data Import

In [3]:
dataset = pd.read_csv("Analysis-of-Healthcare/Datasets/healthcare_dataset.csv")
dataset.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Analysis-of-Healthcare/Datasets/healthcare_dataset.csv'

# Data Preprocessing

## Basic Stats to Understand Data

In [ ]:
dataset.shape

In [ ]:
dataset.info()

* Necessary steps to do:
    - Name column has to be prepared properly
    - Data of Admission & Discharge Date has to be converted to the Date Time type.

In [ ]:
dataset.describe()

### Name Column trasformation

In [ ]:
def normalize_name(name):
    return ' '.join(word.capitalize() for word in name.split())

dataset['Name'] = dataset['Name'].apply(normalize_name)

# Check
dataset['Name'].head()

### Type conversetion of Date related columns

In [ ]:
dataset['Date of Admission'] = pd.to_datetime(dataset['Date of Admission'])
dataset['Discharge Date'] = pd.to_datetime(dataset['Discharge Date'])

#Check
for i in [dataset['Discharge Date'], dataset['Date of Admission']]:
    i.info()

## Missing Value Check

In [ ]:
dataset.isna().sum()

* Clear, 😉

### Data Distribution Analysis

#### Age Feature

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
ax.bar(x = dataset['Age'].value_counts().index, height = dataset['Age'].value_counts())
ax.set_xlabel('Age')
ax.set_ylabel('Frequency')
ax.set_title("Age Distribution")

#### Gender Feature

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
ax.bar(x = dataset['Gender'].value_counts().index, height = dataset['Gender'].value_counts())
ax.set_xlabel('Gender')
ax.set_ylabel('Frequency')
ax.set_title("Gender Distribution")

#### Blood Type

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
ax.bar(x = dataset['Blood Type'].value_counts().index, height = dataset['Blood Type'].value_counts())
ax.set_xlabel('Blood Type')
ax.set_ylabel('Frequency')
ax.set_title("Blood Type Distribution")

#### Medical Condition

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
ax.bar(x = dataset['Medical Condition'].value_counts().index, height = dataset['Medical Condition'].value_counts())
ax.set_xlabel('Medical Condition')
ax.set_ylabel('Frequency')
ax.set_title("Medical Condition Distribution")

#### Doctor Feature

In [ ]:
len(dataset['Doctor'].unique())

#### Hostpital Feature

In [ ]:
len(dataset['Hospital'].unique())

#### Insurance Provider Feature

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
ax.bar(x = dataset['Insurance Provider'].value_counts().index, height = dataset['Insurance Provider'].value_counts())
ax.set_xlabel('Insurance Provider')
ax.set_ylabel('Frequency')
ax.set_title("Insurance Provider Distribution")

#### Room Number Feature

In [ ]:
len(dataset['Room Number'].unique())

#### Admission Type Feature

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
ax.bar(x = dataset['Admission Type'].value_counts().index, height = dataset['Admission Type'].value_counts())
ax.set_xlabel('Admission Type')
ax.set_ylabel('Frequency')
ax.set_title("Admission Type Distribution")

#### Test Results Feature

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
ax.bar(x = dataset['Test Results'].value_counts().index, height = dataset['Test Results'].value_counts())
ax.set_xlabel('Test Results')
ax.set_ylabel('Frequency')
ax.set_title("Test Results Distribution")

#### Medication Feature

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
ax.bar(x = dataset['Medication'].value_counts().index, height = dataset['Medication'].value_counts())
ax.set_xlabel('Medication')
ax.set_ylabel('Frequency')
ax.set_title("Medication Distribution")

In [ ]:
dataset['Age Group'] = pd.cut(dataset['Age'], bins=[0, 20, 40, 60, 100], labels=['0-20', '20-40', '40-60', '61+'])
df = dataset.groupby('Age Group')['Test Results'].agg('count')

f, ax = plt.subplots(figsize=(10, 6))
#ax.bar(x=df.index, height=df)
sns.barplot(x=df.index, y=df, ax=ax, palette='viridis')
ax.set_title("Distribution of Test Results in each Age Group")

- Hastalık Türlerine Göre Ortalama Masraf Analizi:

    Tıbbi durumlara göre ortalama masraf değerlerini hesaplayıp görselleştirin.

In [ ]:
dataset.columns

In [ ]:
df = dataset.groupby('Medical Condition')['Billing Amount'].mean()

f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x=df.index, y=df, ax=ax, palette='rocket_r')
ax.set_title("Average Expense Analysis by Medical Sondition Types")

- Aylık Hasta Kabul Analizi:

    Giriş tarihlerinden ay bilgisi çıkarılarak aylık hasta kabul yoğunluğunu analiz edin.

In [ ]:
months=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
dataset['Month']=pd.to_datetime(dataset['Date of Admission']).dt.month
monthly_counts = dataset['Month'].value_counts().sort_index()

f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x=months, y=monthly_counts, ax=ax, palette='cubehelix')
ax.set_title("Number of Patient per Month")

- Sigorta Sağlayıcılarına Göre Billing Amount Dağılımı:

    Sigorta şirketlerine göre farklı Billing Amount dağılımını gruplandırın.

In [ ]:
df = dataset.groupby(['Insurance Provider', 'Gender'])['Billing Amount'].mean().reset_index().sort_values(['Insurance Provider', 'Gender'])

f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x='Insurance Provider', y='Billing Amount', hue='Gender', data=df, ax=ax, palette="rocket_r")

- Kabul Türüne Göre Hastanede Kalış Süresi Analizi:

    Taburcu ve giriş tarihlerini kullanarak kalış süresini hesaplayın ve giriş türüne göre ortalamalarını karşılaştırın.

In [ ]:
dataset.columns

In [ ]:
dataset['los'] = np.subtract(dataset['Discharge Date'], dataset['Date of Admission']).dt.days

df = dataset.groupby(['Admission Type'])['los'].agg('mean').reset_index().sort_values('Admission Type')

f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(
    data=df,
    x='Admission Type',
    y='los',
    ax=ax,
    palette="magma"
)
ax.set_title("Analysis of Length of Stay by Admission Type")

- Korelasyon Analizi:

    Özellikler arasındaki korelasyonu analiz ederek anlamlı ilişkileri belirleyin.

In [ ]:
dataset=dataset.drop(['Age Group', 'Month', 'los'], axis=1)

In [ ]:
feature_corr = dataset.corr(numeric_only=True)

f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(data=feature_corr, annot=True, cmap="magma")

- Kan Gruplarına Göre Masraf Dağılımı:

    Kan gruplarına göre toplam veya ortalama masrafı analiz edin.

In [ ]:
df = dataset.groupby(['Blood Type'])['Billing Amount'].agg('mean').sort_values(ascending=True)

f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x=df.index, y=df.values, ax=ax, palette='crest')
ax.set_title("Analysis of Blood Type by Billing Amount")

* Billing Amount Accordig to the Blood Type per Gender

In [ ]:
df = dataset.groupby(['Gender', 'Blood Type'])['Billing Amount'].agg('sum').reset_index().sort_values('Blood Type')

f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(data=df, x='Blood Type', y='Billing Amount', hue='Gender', ax=ax, palette='viridis')
ax.set_title("Billing Amount Accordig to the Blood Type per Gender")

# Feature Engineering

* Creating a new field `Admission Duration`:
    - Instead of entrance and exit dates of patient, we can calculate Admissin Duration depending on these dates. After with that calculation, we will not need for `Date Of Admission`, `Discharge Date`, `Name`, `Doctor`, `Hospital` columns so, we can drop them. With this operation we can reduce operation cost.

In [ ]:
dataset['Admission Duration'] = (dataset['Discharge Date'] - dataset['Date of Admission']).dt.days
dataset.drop(['Name', 'Doctor', 'Date of Admission', 'Discharge Date', 'Hospital', 'Room Number'], axis=1, inplace=True)

* Next we need to also convert categorical features into numerical.

In [ ]:
dataset.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
dataset['Gender'] = label_encoder.fit_transform(dataset['Gender'])
dataset['Blood Type'] = label_encoder.fit_transform(dataset['Blood Type'])
dataset['Medical Condition'] = label_encoder.fit_transform(dataset['Medical Condition'])
dataset['Insurance Provider'] = label_encoder.fit_transform(dataset['Insurance Provider'])
dataset['Admission Type'] = label_encoder.fit_transform(dataset['Admission Type'])
dataset['Medication'] = label_encoder.fit_transform(dataset['Medication'])
dataset['Test Results'] = label_encoder.fit_transform(dataset['Test Results'])

In [ ]:
#dataset['Gender'] = dataset['Gender'].map({'Male': 1, 'Female': 0})
#dataset['Test Results'] = dataset['Test Results'].map({'Normal': 0, 'Abnormal': 1, 'Inconclusive': 2})

In [ ]:
# dataset = pd.get_dummies(
    # dataset,
    # columns=[
        # 'Gender',
        # 'Medication',
        # 'Blood Type',
        # 'Admission Type',
        # 'Medical Condition',
        # 'Insurance Provider'
    # ],
    # drop_first=True
# )

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataset[['Age', 'Billing Amount', 'Admission Duration']] = scaler.fit_transform(dataset[['Age', 'Billing Amount', 'Admission Duration']])
dataset.head()

# Modeling

## Base Model: Logistic Regression

* Train - Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset.drop(columns=['Test Results'])
y = dataset['Test Results']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Önem derecesini görselleştirin
feature_importances = model.feature_importances_
plt.figure(figsize=(10, 6))
plt.barh(X.columns, feature_importances)

* `Base Model = Logistic Regression`

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500)
log_reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
y_pred_logreg = log_reg.predict(X_test)

score_logreg = accuracy_score(y_test, y_pred_logreg)
classification_report_logreg = classification_report(y_test, y_pred_logreg)
print(classification_report_logreg)
print("------------------------------------------")
confusion_matrix_logreg = confusion_matrix(y_test, y_pred_logreg)
print(confusion_matrix_logreg)

* According to the report and confusion matrix, we can clearly say that model is working to predict randomly so, ``accuracy is %34``. For a base model, it's too poor.
* Also this consequence can be seen from confusion matrix, many instance of class have been classified wrong.
* The reason why the model is so low that it's because of linear model structure, because our target variable descleares 3 kinds of class.

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

classification_report_rfc = classification_report(y_test, y_pred_rf)
print(classification_report_rfc)
print("_----------------------------------_")
confusion_matrix_rfc = confusion_matrix(y_test, y_pred_rf)
print(confusion_matrix_rfc)
print("_----------------------------------_")
score_rfc = rnd_clf.score(X_test, y_test)
print(score_rfc)

### Hyperparameter Tuning for RFC

In [ ]:
best_rf_model_file = os.path.join('../Models', 'best_rf_model.pkl')

if os.path.exists(best_rf_model_file):
    best_rf_model = joblib.load(best_rf_model_file)
else:
    print("Training has been started...")
    rnd_clf = RandomForestClassifier(random_state=42)

    param_grid = {
        'n_estimators': [50, 100, 200],        # Ağaç sayısı
        'max_depth': [None, 10, 20, 30],       # Maksimum derinlik
        'min_samples_split': [2, 5, 10],       # Düğümlerin bölünmesi için minimum örnek sayısı
        'min_samples_leaf': [1, 2, 4],          # Yaprak düğümdeki minimum örnek sayısı
        'bootstrap': [True, False]
    }

    grid_search = GridSearchCV(estimator=rnd_clf,
                               param_grid=param_grid,
                               cv=5,
                               scoring='accuracy',
                               verbose=2,
                               n_jobs=-1)

    grid_search.fit(X_train, y_train)

    #Saving the best model
    best_rf_model = grid_search.best_estimator_
    joblib.dump(best_rf_model, best_rf_model_file)
    print("Best model has been saved.")

# En iyi parametreler ve skor
if 'grid_search' in locals():
    print("Best parameters: ", grid_search.best_params_)
    print("Best Score: ", grid_search.best_score_)

y_pred_rfc_tuned = best_rf_model.predict(X_test)

# Model Performance Evaluation
score_rfc_tuned = accuracy_score(y_test, y_pred_rfc_tuned)
print("Test Seti Doğruluğu:", score_rfc_tuned)
print("Classification Report:")
classification_report_rfc_tuned = classification_report(y_test, y_pred_rfc_tuned)
print(classification_report_rfc_tuned)
print("Confusion Matrix:")
confusion_matrix_rfc_tuned = confusion_matrix(y_test, y_pred_rfc_tuned)
print(confusion_matrix_rfc_tuned)

## SVM with RBF

* Data is non-linear so, RBF is neceessary.

In [ ]:
from sklearn.svm import SVC

# SVM modeli RBF kernel ile oluşturma ve eğitme
svm_rbf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_rbf.fit(X_train, y_train)

# Tahminler yapma
y_pred_svm = svm_rbf.predict(X_test)

# Model performansını değerlendirme
classification_report_svm = classification_report(y_test, y_pred_svm)
print("Classification Report:")
print(classification_report_svm)
confusion_matrix_svm = confusion_matrix(y_test, y_pred_svm)
print("Confusion Matrix:")
print(confusion_matrix_svm)

### Hyperparameter Tuning for SVM

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],         # Regularization parameter
    'gamma': [1, 0.1, 0.01, 0.001], # RBF kernel için gamma
    'kernel': ['rbf']               # Sadece RBF kernel
}

best_svm_model_file = os.path.join('../Models', 'best_svm_model.pkl')

#Model Check
if os.path.exists(best_svm_model_file):
    best_svm_model = joblib.load(best_svm_model_file)
else:
    print("Training has started...")
    grid = GridSearchCV(
        SVC(random_state=42),
        param_grid,
        refit=True,
        verbose=2,
        cv=3
    )
    grid.fit(X_train, y_train)

    #Saving the best model
    best_svm_model = grid.best_estimator_
    joblib.dump(best_svm_model, best_svm_model_file)

if 'grid' in locals():
    print("Best Parameters:", grid.best_params_)
    print("Best Cross-Validation Score:", grid.best_score_)

# Test seti üzerinde tahminler yapma
y_pred_svm_tuned = best_svm_model.predict(X_test)

# Model performansını değerlendirme
score_svm_tuned = accuracy_score(y_test, y_pred_svm_tuned)
classification_report_svm_tuned = classification_report(y_test, y_pred_svm_tuned)
print("Classification Report:")
print(classification_report_svm_tuned)
confusion_matrix_svm_tuned = confusion_matrix(y_test, y_pred_svm_tuned)
print("Confusion Matrix:")
print(confusion_matrix_svm_tuned)

In [ ]:
# En iyi parametreleri ve en iyi skoru yazdırma
print("Best Parameters:", grid.best_params_)
print("Best Cross-Validation Score:", grid.best_score_)

In [ ]:
# Model performansını değerlendirme
print("Classification Report:")
print(classification_report(y_test, y_pred_svm_tuned))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_svm_tuned))

## XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(random_state=42)
xgb_clf.fit(X_train, y_train)

y_pred_xgb = xgb_clf.predict(X_test)

score_xgb = accuracy_score(y_test, y_pred_xgb)
print("Accuracy: ", score_xgb)
print("---------------------------------------")
classification_report_xgb = classification_report(y_test, y_pred_xgb)
print("Classification Report: ")
print(classification_report_xgb)
print("---------------------------------------")
confusion_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
print("Confusion Matrix: ")
print(confusion_matrix_xgb)

### Hyperparameter Tuning:

In [ ]:
best_xgb_model_file = os.path.join('../Models', 'best_xgb_model.pkl')

#Model Check
if os.path.exists(best_xgb_model_file):
    best_xgb_model = joblib.load(best_xgb_model_file)
else:
    print("Training has started...")

    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0]
    }

    # Grid Search Time
    grid_search = GridSearchCV(
        estimator=XGBClassifier(random_state=42),
        param_grid=param_grid,
        scoring='accuracy',
        cv=5,
        verbose=2,
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)

    #Saving the best model
    best_xgb_model = grid_search.best_estimator_
    joblib.dump(best_xgb_model, best_xgb_model_file)

if 'grid_search' in locals():
    print("Best Parameters:", grid_search.best_params_)
    print("Best Cross-Validation Score:", grid_search.best_score_)

# Test seti üzerinde tahminler yapma
y_pred_xgb_tuned = best_xgb_model.predict(X_test)

# Model performansını değerlendirme
score_xgb_tuned = accuracy_score(y_test, y_pred_xgb_tuned)
print("Classification Report:")
print("-----------------------------------------")
classification_report_xgb_tuned = classification_report(y_test, y_pred_xgb_tuned)
print(classification_report_xgb_tuned)
print("-----------------------------------------")
print("Confusion Matrix:")
print("-----------------------------------------")
confusion_matrix_xgb_tuned = confusion_matrix(y_test, y_pred_xgb_tuned)
print(confusion_matrix_xgb_tuned)

# Metric Visualization

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from io import StringIO

# Classification report string'lerini tanımlama
classification_report_rfc_tuned = '              precision    recall  f1-score   support\n\n           0       0.43      0.43      0.43      4714\n           1       0.42      0.43      0.43      4509\n           2       0.43      0.43      0.43      4652\n\n    accuracy                           0.43     13875\n   macro avg       0.43      0.43      0.43     13875\nweighted avg       0.43      0.43      0.43     13875\n'

classification_report_svm_tuned = '              precision    recall  f1-score   support\n\n           0       0.43      0.45      0.44      4714\n           1       0.42      0.41      0.42      4509\n           2       0.43      0.42      0.43      4652\n\n    accuracy                           0.43     13875\n   macro avg       0.43      0.43      0.43     13875\nweighted avg       0.43      0.43      0.43     13875\n'

classification_report_xgb_tuned = '              precision    recall  f1-score   support\n\n           0       0.39      0.40      0.39      4714\n           1       0.38      0.39      0.38      4509\n           2       0.38      0.37      0.38      4652\n\n    accuracy                           0.38     13875\n   macro avg       0.38      0.38      0.38     13875\nweighted avg       0.38      0.38      0.38     13875\n'

classification_report_logreg = '              precision    recall  f1-score   support\n\n           0       0.34      0.28      0.31      4714\n           1       0.32      0.37      0.34      4509\n           2       0.34      0.36      0.35      4652\n\n    accuracy                           0.34     13875\n   macro avg       0.34      0.34      0.33     13875\nweighted avg       0.34      0.34      0.33     13875\n'

# Raporları bir dictionary olarak saklayalım
classification_reports = {
    "RFC Tuned": classification_report_rfc_tuned,
    "SVM Tuned": classification_report_svm_tuned,
    "XGB Tuned": classification_report_xgb_tuned,
    "LogReg": classification_report_logreg
}

# Her raporu işleyerek anlamlı bir yapıya dönüştürme
processed_reports = {}
for algo_name, report_text in classification_reports.items():
    # Metin raporunu DataFrame'e dönüştür
    df = pd.read_fwf(StringIO(report_text), skiprows=1)
    df.columns = ['class', 'precision', 'recall', 'f1-score', 'support']

    # Sadece "macro avg" satırını çekiyoruz
    macro_avg_row = df[df['class'] == 'macro avg']
    processed_reports[algo_name] = macro_avg_row[['precision', 'recall', 'f1-score']].iloc[0]

# Görselleştirme için hazır veri oluşturma
algorithms = list(processed_reports.keys())
metrics = ['precision', 'recall', 'f1-score']
data = {metric: [processed_reports[algo][metric] for algo in algorithms] for metric in metrics}

# Subplotları oluşturma
fig, axes = plt.subplots(1, len(metrics), figsize=(18, 6), sharey=True)

colors = ['blue', 'green', 'red', 'orange']  # Her algoritma için bir renk

for i, metric in enumerate(metrics):
    axes[i].bar(algorithms, data[metric], color=colors)
    axes[i].set_title(metric.capitalize())
    axes[i].set_ylabel('Score')
    axes[i].set_ylim(0, 1)  # Skor aralığı
    axes[i].set_xticks(range(len(algorithms)))
    axes[i].set_xticklabels(algorithms, rotation=45)

plt.tight_layout()

In [ ]:
score_rfc_tuned

In [ ]:
score_xgb_tuned

In [ ]:
score_svm_tuned

In [ ]:
score_logreg

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from io import StringIO

# Classification report string'lerini tanımlama
classification_report_rfc_tuned = '              precision    recall  f1-score   support\n\n           0       0.43      0.43      0.43      4714\n           1       0.42      0.43      0.43      4509\n           2       0.43      0.43      0.43      4652\n\n    accuracy                           0.43     13875\n   macro avg       0.43      0.43      0.43     13875\nweighted avg       0.43      0.43      0.43     13875\n'

classification_report_svm_tuned = '              precision    recall  f1-score   support\n\n           0       0.43      0.45      0.44      4714\n           1       0.42      0.41      0.42      4509\n           2       0.43      0.42      0.43      4652\n\n    accuracy                           0.43     13875\n   macro avg       0.43      0.43      0.43     13875\nweighted avg       0.43      0.43      0.43     13875\n'

classification_report_xgb_tuned = '              precision    recall  f1-score   support\n\n           0       0.39      0.40      0.39      4714\n           1       0.38      0.39      0.38      4509\n           2       0.38      0.37      0.38      4652\n\n    accuracy                           0.38     13875\n   macro avg       0.38      0.38      0.38     13875\nweighted avg       0.38      0.38      0.38     13875\n'

classification_report_logreg = '              precision    recall  f1-score   support\n\n           0       0.34      0.28      0.31      4714\n           1       0.32      0.37      0.34      4509\n           2       0.34      0.36      0.35      4652\n\n    accuracy                           0.34     13875\n   macro avg       0.34      0.34      0.33     13875\nweighted avg       0.34      0.34      0.33     13875\n'

# Raporları bir dictionary olarak saklayalım
classification_reports = {
    "RFC Tuned": classification_report_rfc_tuned,
    "SVM Tuned": classification_report_svm_tuned,
    "XGB Tuned": classification_report_xgb_tuned,
    "LogReg": classification_report_logreg
}

# Her raporu işleyerek anlamlı bir yapıya dönüştürme
processed_reports = {}
for algo_name, report_text in classification_reports.items():
    df = pd.read_fwf(StringIO(report_text), skiprows=1)
    df.columns = ['class', 'precision', 'recall', 'f1-score', 'support']
    macro_avg_row = df[df['class'] == 'macro avg']
    processed_reports[algo_name] = macro_avg_row[['precision', 'recall', 'f1-score']].iloc[0]

# Ek olarak accuracy skorlarını tanımlama
accuracy_scores = {
    "RFC Tuned": 0.42897297297297273,
    "XGB Tuned": 0.3842162162162162,
    "SVM Tuned": 0.428036036036036,
    "LogReg": 0.3354954954954955
}

# Görselleştirme için veri hazırlama
algorithms = list(processed_reports.keys())
metrics = ['precision', 'recall', 'f1-score']
data = {metric: [processed_reports[algo][metric] for algo in algorithms] for metric in metrics}

# Subplotları oluşturma (4 grafik: precision, recall, f1-score ve accuracy)
fig, axes = plt.subplots(1, len(metrics) + 1, figsize=(24, 6), sharey=False)

colors = ['blue', 'green', 'red', 'orange']  # Her algoritma için renk

# İlk 3 grafiği çizme (precision, recall, f1-score)
for i, metric in enumerate(metrics):
    axes[i].bar(algorithms, data[metric], color=colors)
    axes[i].set_title(metric.capitalize())
    axes[i].set_ylabel('Score')
    axes[i].set_ylim(0, 1)
    axes[i].set_xticks(range(len(algorithms)))
    axes[i].set_xticklabels(algorithms, rotation=45)

# 4. grafik olarak accuracy grafiğini ekleme
axes[3].bar(algorithms, list(accuracy_scores.values()), color=colors)
axes[3].set_title("Accuracy")
axes[3].set_ylabel('Score')
axes[3].set_ylim(0, 1)
axes[3].set_xticks(range(len(algorithms)))
axes[3].set_xticklabels(algorithms, rotation=45)

plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from io import StringIO

# Classification report string'lerini tanımlama
classification_report_rfc_tuned = '              precision    recall  f1-score   support\n\n           0       0.43      0.43      0.43      4714\n           1       0.42      0.43      0.43      4509\n           2       0.43      0.43      0.43      4652\n\n    accuracy                           0.43     13875\n   macro avg       0.43      0.43      0.43     13875\nweighted avg       0.43      0.43      0.43     13875\n'

classification_report_svm_tuned = '              precision    recall  f1-score   support\n\n           0       0.43      0.45      0.44      4714\n           1       0.42      0.41      0.42      4509\n           2       0.43      0.42      0.43      4652\n\n    accuracy                           0.43     13875\n   macro avg       0.43      0.43      0.43     13875\nweighted avg       0.43      0.43      0.43     13875\n'

classification_report_xgb_tuned = '              precision    recall  f1-score   support\n\n           0       0.39      0.40      0.39      4714\n           1       0.38      0.39      0.38      4509\n           2       0.38      0.37      0.38      4652\n\n    accuracy                           0.38     13875\n   macro avg       0.38      0.38      0.38     13875\nweighted avg       0.38      0.38      0.38     13875\n'

classification_report_logreg = '              precision    recall  f1-score   support\n\n           0       0.34      0.28      0.31      4714\n           1       0.32      0.37      0.34      4509\n           2       0.34      0.36      0.35      4652\n\n    accuracy                           0.34     13875\n   macro avg       0.34      0.34      0.33     13875\nweighted avg       0.34      0.34      0.33     13875\n'

# Raporları bir dictionary olarak saklayalım
classification_reports = {
    "RFC Tuned": classification_report_rfc_tuned,
    "SVM Tuned": classification_report_svm_tuned,
    "XGB Tuned": classification_report_xgb_tuned,
    "LogReg": classification_report_logreg
}

# Her raporu işleyerek anlamlı bir yapıya dönüştürme
processed_reports = {}
for algo_name, report_text in classification_reports.items():
    df = pd.read_fwf(StringIO(report_text), skiprows=1)
    df.columns = ['class', 'precision', 'recall', 'f1-score', 'support']
    macro_avg_row = df[df['class'] == 'macro avg']
    processed_reports[algo_name] = macro_avg_row[['precision', 'recall', 'f1-score']].iloc[0]

# Ek olarak accuracy skorlarını tanımlama
accuracy_scores = {
    "RFC Tuned": 0.42897297297297273,
    "XGB Tuned": 0.3842162162162162,
    "SVM Tuned": 0.428036036036036,
    "LogReg": 0.3354954954954955
}

# Görselleştirme için veri hazırlama
algorithms = list(processed_reports.keys())
metrics = ['precision', 'recall', 'f1-score']
data = {metric: [processed_reports[algo][metric] for algo in algorithms] for metric in metrics}

# Subplotları oluşturma (2 satır, 2 sütun)
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()  # 2x2 eksenleri tek bir listeye dönüştür

colors = ['blue', 'green', 'red', 'orange']  # Her algoritma için renk

# İlk 3 grafiği çizme (precision, recall, f1-score)
for i, metric in enumerate(metrics):
    axes[i].bar(algorithms, data[metric], color=colors)
    axes[i].set_title(f"{metric.capitalize()} Scores")
    axes[i].set_ylabel('Score')
    axes[i].set_ylim(0, 1)
    axes[i].set_xticklabels(algorithms, rotation=45)

# 4. grafik: Accuracy
axes[3].bar(algorithms, list(accuracy_scores.values()), color=colors)
axes[3].set_title("Accuracy Scores")
axes[3].set_ylabel('Score')
axes[3].set_ylim(0, 1)
axes[3].set_xticklabels(algorithms, rotation=45)

plt.tight_layout()
